### 0. Import

In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 1. combine mon_features.pkl & unmon_features.pkl into features_df

In [3]:
import pandas as pd
import numpy as np
import pickle

MON_FILE_PATH = '/content/drive/MyDrive/기학/모델 학습/data/mon_features.pkl'
UNMON_FILE_PATH = '/content/drive/MyDrive/기학/모델 학습/data/unmon_features.pkl'

LABEL_COLUMN = ['website_label', 'monitored_label']

FEATURES_VER1 = [
    'total_transmission_time', 'std_inter_packet_time',
    'avg_outgoing_burst_size', 'avg_incoming_burst_size',
    'num_outgoing_packets', 'cumul_max',
    'outgoing_first_30', 'avg_incoming_order_first_30'
]

mon_features_df = pd.read_pickle(MON_FILE_PATH)
unmon_features_df = pd.read_pickle(UNMON_FILE_PATH)

features_df = pd.concat([mon_features_df, unmon_features_df], ignore_index=True)

X = features_df[FEATURES_VER1]
y = features_df[LABEL_COLUMN[0]]


# replace unmonitored label(-1) into 95
y = y.replace({-1: 95})

print(X)
print(y)

       total_transmission_time  std_inter_packet_time  \
0                        10.14               0.041168   
1                        10.16               0.163930   
2                        11.11               0.066661   
3                        13.36               0.047809   
4                        10.64               0.038760   
...                        ...                    ...   
28995                    32.09               0.163669   
28996                    38.62               0.114350   
28997                    34.93               1.331199   
28998                    11.84               0.083521   
28999                     9.62               0.026874   

       avg_outgoing_burst_size  avg_incoming_burst_size  num_outgoing_packets  \
0                     1.551282                16.666667                 121.0   
1                     1.702128                 9.319149                  80.0   
2                     1.552632                16.315789                 

In [4]:
# 로그 변환
log_cols = []

for col in X.columns:
    # 숫자형이고 & 음수가 하나도 없는 경우만 로그 변환 리스트에 추가
    if pd.api.types.is_numeric_dtype(X[col]) and (X[col] < 0).sum() == 0:
        log_cols.append(col)

# 리스트에 있는 컬럼들 로그 변환
if log_cols:
    X[log_cols] = np.log1p(X[log_cols])

/tmp/ipython-input-1747048184.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[log_cols] = np.log1p(X[log_cols])


### 2. model [K-NN]

#### default 학습

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [6]:
# Scaling 추가
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [7]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)

KNeighborsClassifier()

In [8]:
y_pred = knn.predict(X_test)
y_pred_proba = knn.predict_proba(X_test)

In [9]:
acc = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba, multi_class="ovr")
report = classification_report(y_test, y_pred)

print("=== K-NN 기본 성능 ===")
print("Accuracy:", acc)
print("ROC-AUC:", roc_auc)
print("\nClassification Report:\n", report)

=== K-NN 기본 성능 ===
Accuracy: 0.7513793103448276
ROC-AUC: 0.9322982817354258

Classification Report:
               precision    recall  f1-score   support

           0       0.42      0.50      0.45        40
           1       0.67      0.72      0.70        40
           2       0.53      0.68      0.59        40
           3       0.54      0.78      0.64        40
           4       0.66      0.78      0.71        40
           5       0.71      0.75      0.73        40
           6       0.83      0.85      0.84        40
           7       0.67      0.50      0.57        40
           8       0.63      0.68      0.65        40
           9       0.46      0.68      0.55        40
          10       0.62      0.72      0.67        40
          11       0.65      0.60      0.62        40
          12       0.82      0.82      0.82        40
          13       0.54      0.33      0.41        40
          14       0.50      0.60      0.55        40
          15       0.63      0.65 

#### 하이퍼 파라미터 튜닝 후 학습(GridSearchCV 이용)

In [10]:
param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11, 15, 21],
    'weights': ['uniform', 'distance'],
    'metric': ['minkowski', 'manhattan', 'euclidean']
}

grid = GridSearchCV(knn, param_grid, cv=5, scoring='f1_macro')
grid.fit(X_train, y_train)

print(grid.best_params_)
print(grid.best_estimator_)

grid_pred = grid.predict(X_test)
grid_pred_proba = grid.predict_proba(X_test)

{'metric': 'manhattan', 'n_neighbors': 3, 'weights': 'distance'}
KNeighborsClassifier(metric='manhattan', n_neighbors=3, weights='distance')


In [11]:
acc = accuracy_score(y_test, grid_pred)
roc_auc = roc_auc_score(y_test, grid_pred_proba, multi_class="ovr")
report = classification_report(y_test, grid_pred)

print("=== K-NN 튜닝 후 성능 ===")
print("Accuracy:", acc)
print("ROC-AUC:", roc_auc)
print("\nClassification Report:\n", report)

=== K-NN 튜닝 후 성능 ===
Accuracy: 0.7931034482758621
ROC-AUC: 0.929346371480187

Classification Report:
               precision    recall  f1-score   support

           0       0.66      0.53      0.58        40
           1       0.77      0.82      0.80        40
           2       0.62      0.75      0.68        40
           3       0.58      0.82      0.68        40
           4       0.85      0.72      0.78        40
           5       0.85      0.85      0.85        40
           6       0.88      0.88      0.88        40
           7       0.75      0.60      0.67        40
           8       0.75      0.68      0.71        40
           9       0.67      0.75      0.71        40
          10       0.77      0.82      0.80        40
          11       0.77      0.60      0.68        40
          12       0.85      0.88      0.86        40
          13       0.55      0.45      0.49        40
          14       0.60      0.53      0.56        40
          15       0.82      0.78